# INFO 2950 Final Project - Phase II

## Research Questions:
Question:

Insert data sources here:

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

import duckdb

## Data Descriptions:

The data comes from different sources:

 - https://www.electproject.org/election-data/voter-turnout-data
 - https://fred.stlouisfed.org/release/tables?rid=330&eid=391444&od=2009-01-01# 
 - https://www.kff.org/other/state-indicator/distribution-by-raceethnicity/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D
 - https://fred.stlouisfed.org/release/tables?rid=249&eid=259515&od=# 
 - https://www.britannica.com/topic/largest-U-S-state-by-area
 - https://fred.stlouisfed.org/release/tables?rid=118&eid=259194

## Data Cleaning:

Our goal was to have a single table that has a column for each **State**, **Year**, **Voter Turnout**, **Voter Eligibility**, **% Population With Bachelor's Degrees**, **% Population Non-White**, and **Income**. The **Year** column would have each election from 2000 to 2022, every 2 years. We started with the 10 csvs that contain **Turnout**,**Eligibility**, **% Non-Citizen**, and **% Ineligible Felon**.

As a sanity check, since D.C. is included as a state, we should have 51*11 = 561 rows.

In [205]:
#read csv, skip rows 0 and 2 to remove headers and 'United States'
#including thousands to parse commas
election_year_data_2000 = pd.read_csv('data/election_2000.csv', skiprows=[0,2] , thousands=",")
election_year_data_2000['Year'] = 2000

election_year_data_2002 = pd.read_csv('data/election_2002.csv', skiprows=[0,2] , thousands=",")
election_year_data_2002['Year'] = 2002

election_year_data_2004 = pd.read_csv('data/election_2004.csv', skiprows=[0,2] , thousands=",")
election_year_data_2004['Year'] = 2004

election_year_data_2006 = pd.read_csv('data/election_2006.csv', skiprows=[0,2] , thousands=",")
election_year_data_2006['Year'] = 2006

election_year_data_2008 = pd.read_csv('data/election_2008.csv', skiprows=[0,2] , thousands=",")
election_year_data_2008['Year'] = 2008

election_year_data_2010 = pd.read_csv('data/election_2010.csv', skiprows=[0,2] , thousands=",")
election_year_data_2010['Year'] = 2010

election_year_data_2012 = pd.read_csv('data/election_2012.csv', skiprows=[0,2] , thousands=",")
election_year_data_2012['Year'] = 2012

election_year_data_2014 = pd.read_csv('data/election_2014.csv', skiprows=[0,2] , thousands=",")
election_year_data_2014['Year'] = 2014

election_year_data_2016 = pd.read_csv('data/election_2016.csv', skiprows=[0,2] , thousands=",")
election_year_data_2016['Year'] = 2016

election_year_data_2018 = pd.read_csv('data/election_2018.csv', skiprows=[0,2] , thousands=",")
election_year_data_2018['Year'] = 2018

election_year_data_2020 = pd.read_csv('data/election_2020.csv', skiprows=[0,2] , thousands=",")
election_year_data_2020['Year'] = 2020

election_year_data_2022 = pd.read_csv('data/election_2022.csv', skiprows=[0,2] , thousands=",")
election_year_data_2022['Year'] = 2022

election_year_data = pd.concat([election_year_data_2000,
                                election_year_data_2002,
                                election_year_data_2004,
                                election_year_data_2008,
                                election_year_data_2010,
                                election_year_data_2012,
                                election_year_data_2014, 
                                election_year_data_2016, 
                                election_year_data_2018,
                                election_year_data_2020,
                                election_year_data_2022],ignore_index=True)

print(election_year_data.shape)
election_year_data = election_year_data.rename(columns={'Unnamed: 0':'State'})
election_year_data

(561, 17)


,State,VEP Total Ballots Counted,VEP Highest Office,VAP Highest Office,Total Ballots Counted,Highest Office,Voting-Eligible Population (VEP),Voting-Age Population (VAP),% Non-citizen,Prison,Probation,Parole,Total Ineligible Felon,Overseas Eligible,Year,State Abv,VEP Highest Office.1
0,Alabama,NaN,51.6%,50.1%,NaN,1672551.0,3241682,3334576,1.5%,26225,40178,5484,51798,NaN,2000,NaN,NaN
1,Alaska,68.7%,68.1%,65.0%,287825.0,285560.0,419111,440296,3.4%,4173,4779,525,7088,NaN,2000,NaN,NaN
2,Arizona,46.4%,45.6%,40.2%,1559520.0,1532016.0,3357701,3816498,10.3%,26510,59810,3474,59889,NaN,2000,NaN,NaN
3,Arkansas,NaN,47.9%,46.0%,NaN,921781.0,1925961,2001774,2.3%,11915,28409,8659,34779,NaN,2000,NaN,NaN
4,California,56.6%,55.7%,44.1%,11142843.0,10965849.0,19685258,24867252,18.8%,163001,0,117647,280648,NaN,2000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,Virginia,47.6%,NaN,NaN,3021956.0,NaN,6348891,6836463,6.0%,30620,44353,2017,76990,NaN,2022,VA,NaN
557,Washington,55.3%,NaN,NaN,3067686.0,NaN,5543001,6164761,8.8%,15056,49805,14536,79397,NaN,2022,WA,NaN
558,West Virginia,35.4%,NaN,NaN,494753.0,NaN,1396169,1423003,0.9%,6031,4308,3682,14021,NaN,2022,WV,NaN
559,Wisconsin,59.8%,NaN,NaN,2673154.0,NaN,4467396,4655496,3.0%,19846,26486,23174,69506,NaN,2022,WI,NaN


We're now loading in data about the education level of each state in each election year. As a sanity check, we should have 8*51 = 408 rows.

In [57]:
education = pd.read_csv('data/education.csv', thousands=",")
education = education.melt(id_vars=['Name'],value_name="Education",var_name="Year")
print(education.shape)
education

,Name,Year,Education
0,Alabama,2006,21.1
1,Alaska,2006,26.9
2,Arizona,2006,25.5
3,Arkansas,2006,18.2
4,California,2006,29.0
...,...,...,...
403,Virginia,2022,42.2
404,Washington,2022,39.5
405,West Virginia,2022,24.8
406,Wisconsin,2022,33.2


We're now loading in data about the education level of each state in each election year. As a sanity check, we should have 11*51 = 612 rows.

In [60]:
income = pd.read_csv('data/income.csv', thousands=",")
income = income.melt(id_vars=['Name'],value_name="Income",var_name="Year")
print(income.shape)
income

(612, 3)


,Name,Year,Income
0,Alabama,2000,4452.170
1,Alaska,2000,627.963
2,Arizona,2000,5160.590
3,Arkansas,2000,2678.590
4,California,2000,33987.980
...,...,...,...
607,Virginia,2022,8679.100
608,Washington,2022,7784.480
609,West Virginia,2022,1774.040
610,Wisconsin,2022,5890.540


In [83]:
diversity = pd.read_csv('data/diversity.csv', thousands=",", skiprows=[1])
diversity = diversity.melt(id_vars=['Location'],value_name="Percent White",var_name="Year")
diversity['Year'] = diversity['Year'].replace(r'(.*)__.*', r'\1', regex=True)
print(diversity.shape)
diversity

(714, 3)


,Location,Year,Percent White
0,Alabama,2008,0.68670
1,Alaska,2008,0.66160
2,Arizona,2008,0.58220
3,Arkansas,2008,0.75710
4,California,2008,0.42030
...,...,...,...
709,Virginia,2022,0.58904
710,Washington,2022,0.63597
711,West Virginia,2022,0.90436
712,Wisconsin,2022,0.79052


In [159]:
age = pd.read_csv('data/age.csv', thousands=",", skiprows=[0,1,3])

list_years = np.arange(2008,2020)
list_years = np.append(list_years, [2021,2022])

for i in list_years:
    age[str(i)+'__EstMeanAge'] = ((age[str(i)+'__Children 0-18']*9) + (age[str(i)+'__Adults 19-25']*22) + (age[str(i)+'__Adults 26-34']*30)
                                  + (age[str(i)+'__Adults 35-54']*45) + (age[str(i)+'__Adults 55-64']*60) + (age[str(i)+'__65+']*80))
    age[str(i)+'__EstMeanAge'] = (age[str(i)+'__EstMeanAge']) / (age[str(i)+'__Total'])
    

age = age.loc[:,['Location','2008__EstMeanAge','2012__EstMeanAge','2012__EstMeanAge','2014__EstMeanAge',
                  '2016__EstMeanAge','2018__EstMeanAge','2022__EstMeanAge']]
age = age.melt(id_vars=['Location'],value_name="AverageAge",var_name="Year")
age['Year'] = age['Year'].replace(r'(.*)__.*', r'\1', regex=True)
print(diversity.shape)
age

(714, 3)


,Location,Year,AverageAge
0,Alabama,2008,38.331772
1,Alaska,2008,34.782343
2,Arizona,2008,37.247258
3,Arkansas,2008,38.450157
4,California,2008,36.442932
...,...,...,...
359,Washington,2022,40.752680
360,West Virginia,2022,43.466837
361,Wisconsin,2022,41.863187
362,Wyoming,2022,41.322661


We're now loading in data about the education level of each state in each election year. As a sanity check, we should have 11*51 = 612 rows.

In [187]:
population = pd.read_csv('data/population.csv', thousands=",")
population = population.melt(id_vars=['Name'],value_name="Population",var_name="Year")
print(population.shape)

area = pd.read_csv('data/area_by_state.csv', thousands=",", skiprows=[1])
area = area.iloc[0:56,]

population_density = duckdb.sql("""SELECT Name,Year,Population,Area FROM population LEFT JOIN area ON population.Name = area.State""").df()

population_density['PopulationDensity'] = 1000*population_density['Population']/population_density['Area']
population_density

(612, 3)


,Name,Year,Population,Area,PopulationDensity
0,Alaska,2000,627.963,665384,0.943760
1,Arizona,2000,5160.590,113990,45.272305
2,Arkansas,2000,2678.590,53179,50.369319
3,California,2000,33987.980,163695,207.629922
4,Colorado,2000,4326.920,104094,41.567429
...,...,...,...,...,...
607,Maryland,2018,6042.150,12406,487.034499
608,Alabama,2020,5031.860,52420,95.991225
609,Maryland,2020,6173.690,12406,497.637433
610,Alabama,2022,5073.900,52420,96.793209


In [222]:
#merging everything together

election_demographics = duckdb.sql("""SELECT * FROM election_year_data
                                   LEFT JOIN education ON election_year_data.State = education.Name AND election_year_data.Year = education.Year""").df()

election_demographics = duckdb.sql("""SELECT * FROM election_demographics
                                   LEFT JOIN income ON election_demographics.State = income.Name AND election_demographics.Year = income.Year""").df()

election_demographics = duckdb.sql("""SELECT * FROM election_demographics
                                   LEFT JOIN income ON election_demographics.State = income.Name AND election_demographics.Year = income.Year""").df()

election_demographics

,State,VEP Total Ballots Counted,VEP Highest Office,VAP Highest Office,Total Ballots Counted,Highest Office,Voting-Eligible Population (VEP),Voting-Age Population (VAP),% Non-citizen,Prison,Probation,Parole,Total Ineligible Felon,Overseas Eligible,Year,State Abv,VEP Highest Office.1,Name,Year_1,Income
0,Alabama,None,51.6%,50.1%,NaN,1672551.0,3241682,3334576,1.5%,26225,40178,5484,51798,NaN,2000,None,None,Alabama,2000,4452.170
1,Alaska,68.7%,68.1%,65.0%,287825.0,285560.0,419111,440296,3.4%,4173,4779,525,7088,NaN,2000,None,None,Alaska,2000,627.963
2,Arizona,46.4%,45.6%,40.2%,1559520.0,1532016.0,3357701,3816498,10.3%,26510,59810,3474,59889,NaN,2000,None,None,Arizona,2000,5160.590
3,Arkansas,None,47.9%,46.0%,NaN,921781.0,1925961,2001774,2.3%,11915,28409,8659,34779,NaN,2000,None,None,Arkansas,2000,2678.590
4,California,56.6%,55.7%,44.1%,11142843.0,10965849.0,19685258,24867252,18.8%,163001,0,117647,280648,NaN,2000,None,None,California,2000,33987.980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,Washington,43.1%,41.2%,37.1%,2124330.0,2029189.0,4928407,5479396,8.4%,17345,94112,9880,79928,NaN,2014,WA,None,Washington,2014,7057.530
557,West Virginia,42.5%,41.7%,None,597149.0,586034.0,1404206,1429959,0.8%,7137,6523,3550,14405,NaN,2018,WV,None,West Virginia,2018,1805.950
558,Washington,76.2%,4087631,None,4116894.0,NaN,5405804,6034791,9.1%,18777,76672,12222,78536,NaN,2020,WA,75.6%,Washington,2020,7724.570
559,Colorado,58.3%,None,None,2540666.0,NaN,4357176,4642930,5.8%,14990,0,0,14990,NaN,2022,CO,None,Colorado,2022,5841.040
